# De-identifying and Analyzing Census Data

In this project, I aim to de-identify and analyze census data that presents many potential privacy risks. Census statistics are reported at the block level wich means that Releasing *data* that is useful makes ensuring *privacy* very difficult. However, with the use of principles such as generalization and suppression, I will produce a dataset that balances utility and security.

### Data Processing

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
adult = pd.read_csv("adult_with_pii.csv")
adult.head()

,Name,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,Karrie Trusslove,9/7/67,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,6/7/88,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,8/6/91,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,4/6/09,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,9/16/51,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Name            32561 non-null  object
 1   DOB             32561 non-null  object
 2   SSN             32561 non-null  object
 3   Zip             32561 non-null  int64 
 4   Age             32561 non-null  int64 
 5   Workclass       30725 non-null  object
 6   fnlwgt          32561 non-null  int64 
 7   Education       32561 non-null  object
 8   Education-Num   32561 non-null  int64 
 9   Martial Status  32561 non-null  object
 10  Occupation      30718 non-null  object
 11  Relationship    32561 non-null  object
 12  Race            32561 non-null  object
 13  Sex             32561 non-null  object
 14  Capital Gain    32561 non-null  int64 
 15  Capital Loss    32561 non-null  int64 
 16  Hours per week  32561 non-null  int64 
 17  Country         31978 non-null  object
 18  Target

"Name" and "SSN" are identifing atributes wich will be completly removed. "DOB", "Zip", "Race", "Sex", and "Country" are all quasi-identifiers wich will get generalized or suppresed. Finally, the "target" attribute holds confedintial information that will remain unchanged. 

In [4]:
adult_qids = adult.copy()
adult_qids = adult_qids.drop(columns=['Name', 'SSN', 'Age', 'Workclass', 'fnlwgt', 'Education',
        'Martial Status', 'Occupation', 'Relationship', 'Capital Gain', 'Capital Loss', 'Hours per week'])

In [5]:
adult_qids.head()

,DOB,Zip,Education-Num,Race,Sex,Country,Target
0,9/7/67,64152,13,White,Male,United-States,<=50K
1,6/7/88,61523,13,White,Male,United-States,<=50K
2,8/6/91,95668,9,White,Male,United-States,<=50K
3,4/6/09,25503,7,Black,Male,United-States,<=50K
4,9/16/51,75387,13,Black,Female,Cuba,<=50K


### Checking Anonymity

To implement a function to check whether a dataframe satisfies $k$-Anonymity, we loop over the rows; for each row, we query the dataframe to see how many rows match its values for the quasi-identifiers. If the number of rows in any group is less than $k$, the dataframe does not satisfy $k$-Anonymity for that value of $k$, and we return False. 

In [6]:
def duplicates(df): 
    dup_bool = df.duplicated(subset=["DOB","Zip", "Race", "Sex", "Country",], keep=False)
    dup_true = len(list(dup_bool[dup_bool == True]))
    dup_false = len(list(dup_bool[dup_bool == False]))
    
    return dup_true, dup_false


In [7]:
duplicates(adult_qids)

(0, 32561)

This dataset is severly unanaonimized, not a single row has a duplicate in terms of qausi indentifiers. We will use this function as a finaly check.

In [8]:
adult_qids["Combined"] = adult_qids.apply(lambda row: str(row["DOB"]) + str(row["Zip"]) + str(row["Race"]) + str(row["Sex"]) + str(row["Country"]), axis = 1 )

In [9]:
(adult_qids["Combined"].value_counts())

9/7/6764152WhiteMaleUnited-States       1
10/8/5795437WhiteFemaleUnited-States    1
9/12/0230705WhiteMaleUnited-States      1
12/5/5779438WhiteMaleUnited-States      1
3/11/0030932WhiteMaleUnited-States      1
                                       ..
4/3/9726708WhiteMaleUnited-States       1
11/17/0269022WhiteMaleUnited-States     1
8/7/5554003WhiteMaleUnited-States       1
2/1/9613412WhiteFemaleUnited-States     1
5/11/1286153WhiteFemaleUnited-States    1
Name: Combined, Length: 32561, dtype: int64

In [10]:
def isKAnonymized(df, k):
    for elm in list(df["Combined"].value_counts()):
        if elm < k:
            return False
    return True

In [11]:
isKAnonymized(adult_qids, 3)

False

### Anonymizing

The process of modifying a dataset so that it satisfies $k$-Anonymity is accomplished by *generalizing* the data - modifying values to be less specific, and therefore more likely to match the values of other individuals in the dataset. To anonimize quasi-identifier we must order/organize them in a hyerarcical manner. This organization will be very usful because it will indicated the order at wihch we genralize or supress attributes. This hyerrchy will priortize attributes with the most amount of unique values.

In [12]:
for qid in ["DOB", "Zip", "Race", "Sex", "Country"]:
    val = len(pd.unique(adult[qid]))
    print(qid + " " + str(val))

DOB 17309
Zip 27791
Race 5
Sex 2
Country 42


#### Hierarchy: Zip, DOB, Country, Race, Sex

In [13]:
def deZip(lst):
    finlst = []
    for zip in lst: 
        temp = math.ceil((zip/10000))
        finlst.append(temp * 10000)
    return finlst


In [14]:
lst = list(adult_qids["Zip"])
x = deZip(lst)
adult_qids["Zip"] = x
min(adult_qids["Zip"].value_counts())

3138

 Zip has been genralized because the min number of rows with the same zip code is 16, much more that 3 

In [15]:
def deDate(lst):
    fin = []
    for dob in lst:
        dob = dob.split("/")
        year = int(dob[2])
        year = year - (year%10)
        fin.append("01/01/"+ str(year))
    return fin

In [16]:
items = list(adult_qids["DOB"])
m = deDate(items)
adult_qids["DOB"] = m
min(adult_qids["DOB"].value_counts())

1410

In [17]:
adult_qids["DOB"]

0        01/01/60
1        01/01/80
2        01/01/90
3         01/01/0
4        01/01/50
           ...   
32556    01/01/60
32557    01/01/50
32558    01/01/60
32559    01/01/70
32560    01/01/10
Name: DOB, Length: 32561, dtype: object

Dates have also been genralized 

In [18]:
len(adult_qids["Country"].unique())

42

In [19]:
north_AM = ['United-States', 'South', 'Canada']
south_AM = ['Cuba', 'Jamaica', 'Mexico', 'Puerto-Rico', 'Honduras', 'Columbia','Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'Guatemala', 'Peru', 'Trinadad&Tobago', 'Nicaragua']
asia = ['India', 'Iran', 'Philippines', 'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'China', 'Japan', 'Yugoslavia',
        'Outlying-US(Guam-USVI-etc)', 'Vietnam', 'Hong', 'Cambodia']
euro = ['England', 'Germany', 'Italy', 'Poland', 'France', 'Greece', 'Scotland' 'Ireland', 'Hungary',
       'Holand-Netherlands']

In [20]:
cbool = adult_qids["Country"] == "Iran"    # outliers: 3 Cambodian females only 1 black female
adult_qids[cbool]


,DOB,Zip,Education-Num,Race,Sex,Country,Target,Combined
135,01/01/0,80000,14,White,Male,Iran,>50K,1/14/0875436WhiteMaleIran
237,01/01/70,40000,13,White,Male,Iran,>50K,4/26/7538655WhiteMaleIran
348,01/01/90,90000,12,White,Male,Iran,<=50K,11/15/9183640WhiteMaleIran
1054,01/01/0,40000,10,White,Male,Iran,<=50K,12/21/0830974WhiteMaleIran
1387,01/01/60,70000,10,White,Male,Iran,>50K,3/1/6967290WhiteMaleIran
2652,01/01/10,40000,13,White,Male,Iran,>50K,2/11/1039609WhiteMaleIran
3389,01/01/80,30000,16,White,Male,Iran,>50K,4/27/8923468WhiteMaleIran
3390,01/01/80,90000,16,White,Male,Iran,>50K,12/26/8081894WhiteMaleIran
4145,01/01/80,60000,13,White,Female,Iran,>50K,3/15/8653013WhiteFemaleIran
4369,01/01/60,70000,10,White,Female,Iran,<=50K,10/16/6866764WhiteFemaleIran


In [21]:
def deCountry(lst):
    fin_country = []
    for country in lst:
        if country in north_AM:
            fin_country.append("North America")
        elif country in south_AM:
            fin_country.append("South America")
        elif country in asia:
            fin_country.append("Asia")
        elif country in euro:
            fin_country.append("Europe")
        else:
            fin_country.append("N/A")
    return fin_country

In [22]:
natonalties = list(adult_qids["Country"])
v = deCountry(natonalties)
adult_qids["Country"] = v
adult_qids["Country"].value_counts()

North America    29371
South America     1410
Asia               729
N/A                619
Europe             432
Name: Country, dtype: int64

Country has alos been anonymized

In [23]:
adult_qids["Race"].unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo',
       'Other'], dtype=object)

In [24]:
westerners = ["White", "Black"]

In [25]:
def deRace(lst):
    race_fin = []
    for race in lst: 
        if race in westerners :
            race_fin.append("Western-American-Euro")
        elif race == "Other":
            race_fin.append("Amer-Indian-Eskimo")
        else:
            race_fin.append(race)
    return race_fin

In [26]:
races = list(adult_qids["Race"])
r = deRace(races)
adult_qids["Race"] = r
adult_qids["Race"].value_counts()

Western-American-Euro    30940
Asian-Pac-Islander        1039
Amer-Indian-Eskimo         582
Name: Race, dtype: int64

### Validation & Results

In [27]:
adult_qids.head()

,DOB,Zip,Education-Num,Race,Sex,Country,Target,Combined
0,01/01/60,70000,13,Western-American-Euro,Male,North America,<=50K,9/7/6764152WhiteMaleUnited-States
1,01/01/80,70000,13,Western-American-Euro,Male,North America,<=50K,6/7/8861523WhiteMaleUnited-States
2,01/01/90,100000,9,Western-American-Euro,Male,North America,<=50K,8/6/9195668WhiteMaleUnited-States
3,01/01/0,30000,7,Western-American-Euro,Male,North America,<=50K,4/6/0925503BlackMaleUnited-States
4,01/01/50,80000,13,Western-American-Euro,Female,South America,<=50K,9/16/5175387BlackFemaleCuba


In [28]:
# str(row["DOB"]) + str(row["Zip"])
adult_qids["Combined"] = adult_qids.apply(lambda row: str(row["DOB"]) + str(row["Zip"]) + str(row["Race"]) + str(row["Sex"]) + str(row["Country"]), axis = 1 )

In [29]:
validity = list(adult_qids["Combined"].value_counts())
count = 0
for elm in validity:
    if elm < 3:
        count+= 1
print(count)


479


In [30]:
adult_qids.shape

(32561, 8)

In [31]:
duplicates(adult_qids)

(32274, 287)

In [32]:
isKAnonymized(adult_qids, 3)

False

In [33]:
def suppressDF(df,k):
    tmp_df = (df["Combined"].value_counts()).to_frame()
    tmp_df = tmp_df.reset_index()
    records_to_be_dropped = []
    for index, rec in tmp_df.iterrows():
        if rec['Combined'] < 3:
            records_to_be_dropped.append(rec['index'])
            
    new_df = adult_qids[~adult_qids['Combined'].isin(records_to_be_dropped)]
    return new_df
    

In [34]:
supDf = suppressDF(adult_qids, 3)
adult_qids = supDf

In [35]:
adult_qids.shape

(31890, 8)

In [36]:
duplicates(adult_qids)

(31890, 0)

In [37]:
isKAnonymized(adult_qids, 3)

True

### Research Questions

 #### #1

In [38]:
counter = 0
dcounter = 0

for index, row in adult.iterrows():
    if row["Sex"] == "Male":
        counter +=1
        
for index, row in adult_qids.iterrows():
    if row["Sex"] == "Male":
        dcounter +=1


In [39]:
print((counter/ 32561) *100)
print((dcounter/ 31890) *100)

66.92054912318419
67.37848855440576


 #### #2

In [40]:
original = ['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo','Other']

for elm in original:
    target = len(adult[(adult["Race"] == elm) & (adult["Target"] == "<=50K")])
    print(elm, " ",target)
    

White   20699
Black   2737
Asian-Pac-Islander   763
Amer-Indian-Eskimo   275
Other   246


In [41]:
modified = ["Western-American-Euro",'Asian-Pac-Islander', 'Amer-Indian-Eskimo' ]

for elm in modified:
    target = len(adult_qids[(adult_qids["Race"] == elm) & (adult_qids["Target"] == "<=50K")])
    print(elm, " ",target)

Western-American-Euro   23234
Asian-Pac-Islander   587
Amer-Indian-Eskimo   343


 #### #3

In [42]:
adult['Education-Num'].unique()

array([13,  9,  7, 14,  5, 10, 12, 11,  4, 16, 15,  3,  6,  2,  1,  8],
      dtype=int64)

In [43]:
educated = [13, 14, 15, 16]
target = 0
under = 0

for index, row in adult.iterrows():
    if row['Education-Num'] in educated:
        target = len(adult[adult["Target"] == "<=50K"])
        under = len(adult[adult["Target"] == ">=50K"])
        

In [44]:
print(target, under)

24720 0


In [45]:
educated = [13, 14, 15, 16]
target = 0
under = 0

for index, row in adult_qids.iterrows():
    if row['Education-Num'] in educated:
        target = len(adult_qids[adult_qids["Target"] == "<=50K"])
        under = len(adult_qids[adult_qids["Target"] == ">=50K"])

In [46]:
print(target, under)

24164 0


 #### #4

In [47]:
counter = 0
dcounter = 0

for index, row in adult.iterrows():
    if row["Sex"] == "Female" and row["Country"] == "Cambodia":
        counter +=1
        
for index, row in adult_qids.iterrows():
     if row["Sex"] == "Female" and row["Country"] == "Asia":
        dcounter +=1

In [49]:
print((counter/ 32561) *100)
print((dcounter/ 31890) *100)

0.009213476244587083
0.46723110693007214


#### #5

In [50]:
counter = 0
dcounter = 0

for index, row in adult.iterrows():
    if row["Sex"] == "Female" and row["Country"] == "Iran":
        counter +=1
        
for index, row in adult_qids.iterrows():
     if row["Sex"] == "Female" and row["Country"] == "Asia":
        dcounter +=1

In [51]:
print((counter/ 32561) *100)
print((dcounter/ 31890) *100)

0.02456926998556555
0.46723110693007214
